## Import libraries and dependencies

In [1]:

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv
load_dotenv()

True

## Define and test the LLM

In [2]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=os.environ["GOOGLE_API_KEY"], temperature=0.5, convert_system_message_to_human=True)

In [3]:
test_query1 = llm.invoke('Define attention mechanism')
print(test_query1.content)

**Attention Mechanism**

In deep learning, an attention mechanism is a neural network technique that allows a model to selectively focus on specific parts of an input sequence or data structure. It helps the model prioritize relevant information and suppress irrelevant or less important information.

**How it Works:**

1. **Query:** The model generates a query vector, which represents the current state of its attention.
2. **Key-Value Pairs:** The input sequence or data structure is represented as a set of key-value pairs. Each key corresponds to a specific item in the sequence, and the value contains the associated information.
3. **Attention Scores:** The query vector is compared to each key using a scoring function (e.g., dot product, cosine similarity). This calculates an attention score for each key-value pair.
4. **Weighted Sum:** The attention scores are normalized to create a probability distribution. These probabilities are used to weight the values associated with each key.
5

In [4]:
test_query2 = llm.invoke("Write a poem on football")
print(test_query2.content)

**Ode to Football**

On verdant fields where passion reigns,
A leather sphere, a game that sustains.
Twenty-two warriors, hearts ablaze,
In pursuit of glory, their spirits amaze.

The whistle blows, the crowd erupts,
As players charge, with thunderous struts.
A ballet of motion, swift and bold,
Where skill and teamwork unfold.

Dribbling past foes with ease and grace,
Passing with precision, setting the pace.
Shots thunderous, soaring through the air,
Goalkeepers valiant, beyond compare.

The pitch a canvas, where dreams take flight,
Where heroes are born, in the heat of the fight.
Tactics and strategy, a chessboard's game,
Where coaches guide, and players claim.

The roar of the crowd, a symphony grand,
As they cheer their team, hand in hand.
Victories celebrated, defeats embraced,
In this sport where passions are interlaced.

Football, a game that unites and inspires,
A symbol of unity, that never tires.
May its spirit forever burn bright,
A testament to human strength and might.


## Load and explore the CSV file

In [5]:
loader = CSVLoader('dermatology_q&a.csv', source_column='questions')
data = loader.load()

In [6]:
print(data)

[Document(page_content='questions: What is psoriasis and what are its common symptoms?\nanswer: Psoriasis is a chronic autoimmune condition that results in the overproduction of skin cells. This overproduction leads to patches of thick, red skin covered with silvery scales. Common symptoms include red patches of skin covered with thick, silvery scales, small scaling spots (commonly seen in children), dry and cracked skin that may bleed, itching, burning, or soreness, thickened, pitted, or ridged nails, and swollen and stiff joints.', metadata={'source': 'What is psoriasis and what are its common symptoms?', 'row': 0}), Document(page_content="questions: What is the etiology of acne?\nanswer: Acne is primarily caused by the overproduction of oil; blocked hair follicles that don't allow oil to leave the pore, which may cause a clogged pore; bacteria called Propionibacterium acnes that can grow inside the hair follicles and cause inflammation; and hormonal changes, particularly during pube

## Embeddings and Vector Database

Embeddings are the vector representations of words. As the models do not directly understand words and sentences, embeddings are essential for converting textual data into numerical form. These vectors capture semantic meaning and context. Note that similar words reside in the same vector space, reflecting their semantic similarity, which is quantified by cosine similarity. The embeddings are then stored in a vector database, enabling users to perform similarity searches. When a query is made, the vector database extracts similar vectors that match the query. There are several vector databases available for this purpose, including ChromaDB, FAISS, Milvus, and others.

In [7]:
embeddings = HuggingFaceEmbeddings()

vectordb = FAISS.from_documents(documents=data, embedding=embeddings)

In [8]:
# converted the vector database into a retriever object, which allows us to retrieve relevant documents based on similarity between user query and the data in csv.
retriever = vectordb.as_retriever()
relevant_docs = retriever.get_relevant_documents("explain about melanoma?")
relevant_docs

[Document(page_content="questions: What are the symptoms of melanoma?\nanswer: Melanoma signs include a new, unusual growth or a change in an existing mole. Melanomas can develop anywhere on your body, most often in areas that have had exposure to the sun. Common symptoms follow the ABCDE rule: Asymmetry (one half of the mole doesn't match the other), Border irregularity (the edges are ragged or notched), Color variations (having a variety of colors is a warning sign), Diameter (melanomas are usually larger in diameter than the eraser on your pencil, but they may be smaller when first detected), and Evolving (any change in size, shape, color, elevation, or another trait, or any new symptom such as bleeding, itching or crusting, may signal melanoma).", metadata={'source': 'What are the symptoms of melanoma?', 'row': 17}),
 Document(page_content="questions: Can you tell me about the treatment modalities for melanoma?\nanswer: Melanoma treatment depends on the stage and location of the di

## Prompt Template

A prompt template simply refers to the instructions for the language model. By structuring prompts, we can direct the model to generate answers that align with our requiremets and avoid the model from hallucinating.

In [9]:
prompt_template = """Given the following context and a question, generate an answer based on this context only.
    In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
    If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

    CONTEXT: {context}

    QUESTION: {question}"""

In [10]:
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

### RetrievalQA

The RetrievalQA chain first does a retrieval step to fetch relevant documents, then passes those documents into an LLM to generate a response.

In [11]:
chain = RetrievalQA.from_chain_type(
    chain_type="stuff",
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [12]:
chain.invoke("What causes melanoma and what are the symptoms of melanoma?")

{'query': 'What causes melanoma and what are the symptoms of melanoma?',
 'result': "Melanoma occurs when something goes wrong in the melanocytes, the cells that produce melanin — the pigment that gives your skin, hair and eyes color. The exact cause isn't clear, but exposure to ultraviolet (UV) radiation from sunlight or tanning lamps and beds increases your risk of developing melanoma. Other factors that can increase your risk include having many moles or abnormal moles, a family history of melanoma, fair skin, a history of sunburn, living closer to the equator or at a higher elevation, and having a weak immune system.\n\nMelanoma signs include a mole that changes in color, size or feel, or that bleeds. Other symptoms can include a small lesion with an irregular border and portions that appear red, white, blue or blue-black; a painful lesion that itches or burns; a large brownish spot with darker speckles; and dark lesions on your palms, soles, fingertips or toes, or on mucous membra

In [13]:
chain.invoke("tell about the treatment for psoriasis")

{'query': 'tell about the treatment for psoriasis',
 'result': 'Psoriasis treatment aims to stop skin cells from growing so quickly and to remove scales. Options include topical corticosteroids to reduce inflammation and itching, Vitamin D analogues to slow skin cell growth, retinoids to decrease inflammation, and moisturizers to smooth the skin and reduce dryness and scaling.',
 'source_documents': [Document(page_content="questions: What is the treatment for psoriasis?\nanswer: Psoriasis treatment aims to stop skin cells from growing so quickly and to remove scales. Options include topical corticosteroids to reduce inflammation and itching, Vitamin D analogues to slow skin cell growth, retinoids to decrease inflammation, and moisturizers to smooth the skin and reduce dryness and scaling. \n\nFor moderate to severe psoriasis, light therapy may be used. This involves exposing the skin to ultraviolet light on a regular basis and under medical supervision. \n\nSystemic medications are pre

In [14]:
chain.invoke("tell about ichthyosis")

{'query': 'tell about ichthyosis',
 'result': "I don't know.",
 'source_documents': [Document(page_content="questions: What are the common symptoms of vitiligo?\nanswer: Vitiligo is a disease that causes the loss of skin color in blotches. The extent and rate of color loss are unpredictable. Common symptoms include:\n\n1. Patchy loss of skin color, which usually first appears on the hands, face, and areas around body openings and the genitals.\n2. Premature whitening or graying of the hair on your scalp, eyelashes, eyebrows or beard.\n3. Loss of color in the tissues that line the inside of your mouth and nose (mucous membranes).\n4. Loss of or change in color of the inner layer of the eyeball (retina).\n\nIt's important to note that vitiligo is not painful and does not have significant health consequences; however, it can have emotional and psychological impact.", metadata={'source': 'What are the common symptoms of vitiligo?', 'row': 63}),
  Document(page_content='questions: What are 

In [16]:
chain.invoke("tell about the common ways to treat acne and psoriasis")

{'query': 'tell about the common ways to treat acne and psoriasis',
 'result': "I don't know. The provided context does not mention anything about acne.",
 'source_documents': [Document(page_content="questions: What are common treatments for psoriasis?\nanswer: Treatment for psoriasis aims to help control symptoms and prevent complications. It can include:\n\n1. Topical treatments: These are creams and ointments applied to your skin, and are usually the first treatment to try when diagnosed with psoriasis.\n\n2. Light therapy: This treatment involves exposing your skin to ultraviolet light under medical supervision.\n\n3. Oral or injected medications: These are used for severe psoriasis or psoriasis that resists other treatments. They include retinoids, methotrexate, cyclosporine, biologics, and newer oral treatments such as apremilast.\n\nAs psoriasis is a chronic condition, it's important to work with a healthcare provider to manage it effectively.", metadata={'source': 'What are com

In [17]:
chain.invoke("common ways to treat acne")

{'query': 'common ways to treat acne',
 'result': 'Acne treatments aim to reduce oil production, speed up skin cell turnover, fight bacterial infection, or reduce inflammation. Over-the-counter treatments often contain salicylic acid or benzoyl peroxide. Prescription treatments can include topical retinoids, topical antibiotics, combined topical products, oral antibiotics, oral contraceptives for females, anti-androgen agents for females, and isotretinoin for severe acne. In-office procedures, such as light therapy, chemical peels, and extraction of whiteheads and blackheads, can also be beneficial.',
 'source_documents': [Document(page_content="questions: What are the common treatments for acne?\nanswer: Acne treatments aim to reduce oil production, speed up skin cell turnover, fight bacterial infection, or reduce inflammation. Over-the-counter treatments often contain salicylic acid or benzoyl peroxide. Prescription treatments can include topical retinoids, topical antibiotics, combi